# Data cleaning - AI safety for b-tagging@CMS edition
This introduction will explain the different steps inside the `clean.py` script that produces the input and target data to be used for all subsequent studies.

<div align="center">
<img src="https://miro.medium.com/max/3600/1*6NZejVJz5nmxgHeWWTc2Iw.png" width="400"/><br>
    Fig. 1 <a href="https://towardsdatascience.com/data-cleaning-in-python-the-ultimate-guide-2020-c63b88bf0a0d">[from towardsdatascience.com]</a>
</div>

The tutorial is split into four parts:
- [Prerequisites](#prerequisites)
- [Basics](#basics)
- [Applying the cleandataset function interactively](#interactive)
- [Practical data cleaning @HPC](#atHPC)

## Prerequisites <a name="prerequisites"></a>
It is assummed that you have your account, conda environment and grid proxy setup. Missing packages should be installed to make use of the full functionality, my personal environment is [listed here](https://git.rwth-aachen.de/annika-stein/ai_safety_2021/-/blob/master/my-env_october2021.yml).

To understand a bit better how the data cleaning integrates into the whole framework, my thesis (in particular section 2.2) provides more theoretical aspects and the motivation for each step.

## Basics <a name="basics"></a>
We start with a rather technical explanation of the different steps, later there will be some more interactive steps before you will actually execute the whole script at once for all input files. The basics are just meant to show what the code does, instructions for actually executing the code come afterwards.

### Libraries, parser
Look at the beginning of the file `clean.py`: it starts with a couple of imports and introduces a parser with which one can read user-specified arguments to use them during the execution of the code.
```python
import uproot4 as uproot
import numpy as np
import awkward1 as ak

import gc

import json


### Parser ###

import argparse
import ast


parser = argparse.ArgumentParser(description="Perform step 1 of data cleaning")
parser.add_argument("startfile", type=int, help="Number of the file with which cleaning starts")
parser.add_argument("endfile", type=int, help="Number of the file with with cleaning ends")
parser.add_argument("default", type=float, help="default value relative to the minimum of the distribution, with positive sign")
args = parser.parse_args()

startindex = args.startfile
endindex = args.endfile
default = args.default


```
If you use the newest version of uproot (and awkward, I guess as well), then you might adjust the imports a bit.<br>
Using gc one can collect garbage if one needs to free some memory when the automatic behaviour is not enough.<br>
The json package is necessary to load the file paths (see below).<br>

Ignore the `import ast` line, it was used in an earlier version to read in the user input and create a list out of strings automatically.<br>
More important is `argparse`: first, the parser is created with in total three arguments. The types vary, some are integer numbers, but there can also be floating point numbers. Each argument gets an identifier (e.g. startfile) under which the user input can be accessed later. For example, to read the startfile and store the value in a variable (explanation follows later), one needs to parse the arguments and use the respective ones with the dot-notation. In this particular case, the user would call the script with a command like this one: `python clean.py 0 10 0.001` and the values would be available for all subsequent steps.

### Loading file paths and defaults
Next, the script localizes and loads all the paths for the original input files of a given type (e.g. in the example below, only the semileptonic tt̅ events are used) and loads the defaults, which will be put as placeholders whenever the values are not valid.
```python
### The original root-files ###

list_paths = []
with open('/home/um106329/aisafety/new_march_21/qcd_file_paths.json') as json_file:
    json_paths = json.load(json_file)
    '''
    # QCD files
    for entry in json_paths['QCD_HT300to500_TuneCP5_13TeV-madgraph-pythia8']:
        list_paths.append(entry)
    for entry in json_paths['QCD_HT500to700_TuneCP5_13TeV-madgraph-pythia8']:
        list_paths.append(entry)
    for entry in json_paths['QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8']:
        list_paths.append(entry)
    for entry in json_paths['QCD_HT1000to1500_TuneCP5_13TeV-madgraph-pythia8']:
        list_paths.append(entry)
    for entry in json_paths['QCD_HT1500to2000_TuneCP5_13TeV-madgraph-pythia8']:
        list_paths.append(entry)
    for entry in json_paths['QCD_HT2000toInf_TuneCP5_13TeV-madgraph-pythia8']:
        list_paths.append(entry)
    '''  
    # TTtoSemileptonic files
    for entry in json_paths['TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8']:
        list_paths.append(entry)


##################### Selecting default values ####################
minima = np.load('/home/um106329/aisafety/april_21/from_Nik/default_value_studies_minima.npy') #np.zeros(100)for constant default of -999 #have to be determined externally, otherwise they would not be the same for every iteration
defaults_per_variable = minima - default


```
To use the script, you need to first of all download the [json-file that contains the file paths](https://raw.githubusercontent.com/andrzejnovak/nanocc/master/metadata/v2x17.json), download the [numpy-array that contains the defaults](https://git.rwth-aachen.de/nikolas/ai-safety-default-value-studies/-/blob/master/default_value_studies/minima.npy), and then update the paths to your personal directories at the HPC. When you later on start with the semileptonic tt̅ events, just leave the block as it is, if you instead clean the QCD samples, invert the uncommented lines to only put the QCD entries into the list of paths, but not the semileptonic tt̅ events.

Regarding the default values, Nikolas Frediani [calculated them](https://git.rwth-aachen.de/nikolas/ai-safety-default-value-studies/-/blob/master/default_value_studies/prototyping_minima.ipynb) based on the minima of a couple of inputs (this is actually what you can load as `.npy`). But the minima are shifted a tiny bit to place the actual defaults slightly below the minima, such that they will not interfer that much with the bulk distribution. This is what gets specified by the variable called `default`, here and also in the other scripts that follow. The "default" default value is 0.001, this places the actual default values slightly below the minima.

### The "cleandataset" function
With the function specified below, every dataset will be processed to store only the relevant information and perform the actual cleaning. Ignore the old version of the function, which is commented out in between the = and - rows.
```python
### Perform cleaning and save files ###

# =============================================================================================================
# Attention: old version, deprecated
# ...
# ... a lot of old code you can ignore
# ...
# --------------------------------------------------------------------------------------------------------


def cleandataset(f):
    #print('Doing cleaning, isMC = ',isMC)
    feature_names = [k for k in f['Events'].keys() if  (('Jet_eta' == k) or ('Jet_pt' == k) or ('Jet_DeepCSV' in k))]
    # tagger output to compare with later and variables used to get the truth output
    feature_names.extend(('Jet_btagDeepB_b','Jet_btagDeepB_bb', 'Jet_btagDeepC','Jet_btagDeepL'))
    feature_names.extend(('Jet_nBHadrons', 'Jet_hadronFlavour'))
    #print(feature_names)
    #print(len(feature_names))
    
    # go through a specified number of events, and get the information (awkward-arrays) for the keys specified above
    for data in f['Events'].iterate(feature_names, step_size=f['Events'].num_entries, library='ak'):
        break
        
    
    # creating an array to store all the columns with their entries per jet, flatten per-event -> per-jet
    datacolumns = np.zeros((len(feature_names)+1, len(ak.flatten(data['Jet_pt'], axis=1))))
    #print(len(datacolumns))

    for featureindex in range(len(feature_names)):
        a = ak.flatten(data[feature_names[featureindex]], axis=1) # flatten along first inside to get jets
        
        datacolumns[featureindex] = ak.to_numpy(a)

    nbhad = ak.to_numpy(ak.flatten(data['Jet_nBHadrons'], axis=1))
    hadflav = ak.to_numpy(ak.flatten(data['Jet_hadronFlavour'], axis=1))

    target_class = np.full_like(hadflav, 3)                                                      # udsg
    target_class = np.where(hadflav == 4, 2, target_class)                                       # c
    target_class = np.where(np.bitwise_and(hadflav == 5, nbhad > 1), 1, target_class)            # bb
    target_class = np.where(np.bitwise_and(hadflav == 5, nbhad <= 1), 0, target_class)           # b, lepb

    #print(np.unique(target_class))

    #datacolumns[len(feature_names)] = ak.to_numpy(target_class)
    datacolumns[len(feature_names)] = target_class
    #print(np.unique(datacolumns[len(feature_names)]))
        
    datavectors = datacolumns.transpose()
    #print(np.unique(datavectors[:,len(feature_names)]))
    
    #print(i)
    for j in range(67):
        datavectors[:, j][datavectors[:, j] == np.nan]  = defaults_per_variable[j]
        datavectors[:, j][datavectors[:, j] <= -np.inf] = defaults_per_variable[j]
        datavectors[:, j][datavectors[:, j] >= np.inf]  = defaults_per_variable[j]
        datavectors[:, j][datavectors[:, j] == -999]  = defaults_per_variable[j]  # this one line is new and the reason for that is that there can be "original" -999 defaults in the inputs that should now also move into the new
                                                               # default bin, it was not necessary in my old clean_1_2.py code, because I could just leave them where they are, here they need to to be modified
        #print(np.unique(datavectors[:,-1]))
    #print(np.unique(datavectors[:,-1]))
    datavecak = ak.from_numpy(datavectors)
    #print(ak.unique(datavecak[:,-1]))
    #print(len(datavecak),"entries before cleaning step 1")
    
    # cutting stuff away only needs to be done for train, val, test without ScaleFactor framework, there these can just be commented out
    # tracker acceptance and pt range
    datavecak = datavecak[datavecak[:, 0] >= -2.5]
    datavecak = datavecak[datavecak[:, 0] <= 2.5]
    datavecak = datavecak[datavecak[:, 1] >= 20.]
    datavecak = datavecak[datavecak[:, 1] <= 1000.]
    datavecak = datavecak[datavecak[:, 57] <= 0.3]
    #datavecak = datavecak[datavecak[:, 67] >= 0.]
    #datavecak = datavecak[datavecak[:, 67] <= 1.]
    #datavecak = datavecak[datavecak[:, 68] >= 0.]
    #datavecak = datavecak[datavecak[:, 68] <= 1.]
    #datavecak = datavecak[datavecak[:, 69] >= 0.]
    #datavecak = datavecak[datavecak[:, 69] <= 1.]
    #datavecak = datavecak[datavecak[:, 70] >= 0.]
    #datavecak = datavecak[datavecak[:, 70] <= 1.]

    

    # check jetNSelectedTracks, jetNSecondaryVertices > 0
    #datavecak = datavecak[(datavecak[:, 63] > 0) | (datavecak[:, 64] > 0)]  # keep those where at least any of the two variables is > 0, they don't need to be > 0 simultaneously
    #print(len(datavecak),"entries after cleaning step 1")

    alldata = ak.to_numpy(datavecak)
    #print(np.unique(alldata[:,-1]))
        
    
    for track0_vars in [6,12,22,29,35,42,50]:
        alldata[:,track0_vars][alldata[:,64] <= 0] = defaults_per_variable[track0_vars]
    for track0_1_vars in [7,13,23,30,36,43,51]:
        alldata[:,track0_1_vars][alldata[:,64] <= 1] = defaults_per_variable[track0_1_vars]
    for track01_2_vars in [8,14,24,31,37,44,52]:
        alldata[:,track01_2_vars][alldata[:,64] <= 2] = defaults_per_variable[track01_2_vars]
    for track012_3_vars in [9,15,25,32,38,45,53]:
        alldata[:,track012_3_vars][alldata[:,64] <= 3] = defaults_per_variable[track012_3_vars]
    for track0123_4_vars in [10,16,26,33,39,46,54]:
        alldata[:,track0123_4_vars][alldata[:,64] <= 4] = defaults_per_variable[track0123_4_vars]
    for track01234_5_vars in [11,17,27,34,40,47,55]:
        alldata[:,track01234_5_vars][alldata[:,64] <= 5] = defaults_per_variable[track01234_5_vars]
    alldata[:,18][alldata[:,65] <= 0] = defaults_per_variable[18]
    alldata[:,19][alldata[:,65] <= 1] = defaults_per_variable[19]
    alldata[:,20][alldata[:,65] <= 2] = defaults_per_variable[20]
    alldata[:,21][alldata[:,65] <= 3] = defaults_per_variable[21]

    for AboveCharm_vars in [41,48,49,56]:
        alldata[:,AboveCharm_vars][alldata[:,AboveCharm_vars]==-1] = defaults_per_variable[AboveCharm_vars] 
    
    
    datacls = [i for i in range(0,67)]
    datacls.append(73)
    dataset = alldata[:, datacls]
     
    DeepCSV_dataset = alldata[:, 67:71]
    
    return dataset, DeepCSV_dataset
```
First, the features are defined that will be used as inputs, to define the targets (or to store DeepCSV outputs for comparison). Those are all the available keys that start with `Jet_DeepCSV` and the jet pseudorapidity and transverse momentum as inputs, then there are the `Jet_btag` keys which are the DeepCSV outputs and with the number of b hadrons and the hadron flavour one can categorize the jet on its flavour (truth).<br>
With the following loop the events are read in with the awkward library.<br>
This will have all the inputs, but not in the format that will be used for the training. Therefore a new array is build with zeros as placeholders. The dimension on the first axis is one more than the number of features extracted from the oroginal files, because the last column will store the manually generated truth outputs. On the next axis, there will be enough rows to store all jets that are part of the dataset, for that one needs to know the length of the flat array (flat means reducing the depth from Events/jets to only jets, because the information on events will not be needed.)<br>
The placeholders (except for the last column, which comes later) will be filled in the next loop. Again, this needs flat arrays to get jets, and then these are casted to numpy arrays which then fill a specific column.<br>
After this loop, the targets are defined, first by getting the number of b hadrons and the hadron flavour, and then filling a new placeholder array called `target_class` with one of the four values 0,1,2,3 (for b. bb. c or udsg jets). For the definitions see section 2.2.3 of the thesis.<br>
One can again ignore what's commented out, next up the targets are stored in the last column of the big array. This is then transposed such that the definition row and column make a bit more sense, they were flipped around at the beginning.<br>
There is now four cases that automatically require a default value for the entries, no matter what variable one is looking at: if the entry (a specific feature of a given jet) is either plus/minus infinity or not-a-number (nan) or -999, this particular value will be replaced with a default value that corresponds to the specific feature. The array indexing first of all grabs the *j*-th column (feature) for all jets at once, and selects those jets where this *j*-th column has one of the bad values. The >= and <= for the +/- infinity might look a bit strange, but I read somewhere that this was a safe way to get those "numbers" no matter how they are actually stored (no number is "exactly" infinity).<br>
The next switch to awkward arrays might not be necessary anymore, but anyway I did not bother to change it. The datavectors (as ak-arrays) are now cleaned from whole jets (deleting complete rows of the data set). Note the structural difference compared to the "cleaning" steps above, where the jets were not dropped entirely. Now, the array itself becomes smaller with every new cut. The first two lines of these steps select only those jets which are inside the tracker acceptance (cut on jet_eta), additionally, the jet_pt is required to be between 20 and 1000 GeV. The fifth cut in this block is related to the angular distance between the summed four-momentum of the tracks and the jet axis (should be below 0.3).<br>
For some reason, I now transform the array back into numpy-format.<br>
The tricky part is following, with six for loops and more cleaning afterwards related to track variables. The idea is that based on the numbeer of selected tracks, one can only store as many track variables as are present in the selection from the reconstruction algorithm. In every for-loop, the variable jetNSelectedTracks (index: 64 = the 65-th variable) is used to find out if there are enough tracks in the jet to have a meaningful value for the track variable under investigation. `track0_vars` for example list all the variables that are stored for the first track (index 0 = first track, note the zero-indexing everywhere). Line 243 can be read as: look at a specific first-track variable (e.g. index 6) and select all jets that have less than 1 selected tracks (less than one = <= 0, again to be reminded of the zero-indexing, which made writing the code a bit easier for me). After selecting all those "bad" jets, set the value for the given column to the default value for that column. In the next for loop, something similar happens for all second-track variables, so you need at least two selected tracks, otherwise it's a default value, and so on. There is one track variable which depends instead on another "number of things", namely the eta_rel variable. For this one, there are only four tracks for which the variable could exist in the dataset, and the defining variable to check for has index 65, its name is `Jet_DeepCSV_jetNTracksEtaRel` and counts for how many tracks this pseudorapidity between the track and the jet axis has been stored. But the idea is the same, it's just written without a loop because there is only one feature that relies on this number.<br>
There is one more type of input feature which needs our attention regarding defaults: the variables which are given for the first track above the charm mass threshold might come with -1 values as defaults, and we want all defaults to be outside the bulk of the distribution. So for these variables as well, one checks for this specific -1 value and sets this to the respecitve manually computed default instead.<br>

Now the remaining part is just used to split the big array into the inputs+targets or into DeepCSV, for comparison. Inputs and targets are the first 67 features (in) and the last one (out), DeepCSV is part of the remaining columns 67 (inclusive) to 71 (exclusive). Those two arrays will be returned by the function.

### The main loop that runs over several data sets
The function itself would not help much, if one would not call it somewhere: this is finally done in the following block, a loop that runs over chunks of 50 original input files at once.
```python
for innerstart in np.arange(startindex, endindex, 50):
    if innerstart + 49 <= endindex:
        for i,n in enumerate(range(innerstart, innerstart + 50)):
            #print(n)
            if i == 0:
                dataset, DeepCSV_dataset = cleandataset(uproot.open(list_paths[n]))
            else:
                datasetNEW, DeepCSV_datasetNEW = cleandataset(uproot.open(list_paths[n]))
                dataset, DeepCSV_dataset = np.concatenate((dataset, datasetNEW)), np.concatenate((DeepCSV_dataset, DeepCSV_datasetNEW))
        
        np.save(f'/hpcwork/um106329/may_21/cleaned_TT/inputs_{innerstart}_to_{innerstart+49}_with_default_{default}.npy', dataset)  
        np.save(f'/hpcwork/um106329/may_21/cleaned_TT/deepcsv_{innerstart}_to_{innerstart+49}_with_default_{default}.npy', DeepCSV_dataset) 
    else:
        for i,n in enumerate(range(innerstart, endindex)):
            #print(n)
            if i == 0:
                dataset, DeepCSV_dataset = cleandataset(uproot.open(list_paths[n]))
            else:
                datasetNEW, DeepCSV_datasetNEW = cleandataset(uproot.open(list_paths[n]))
                dataset, DeepCSV_dataset = np.concatenate((dataset, datasetNEW)), np.concatenate((DeepCSV_dataset, DeepCSV_datasetNEW))
        
        np.save(f'/hpcwork/um106329/may_21/cleaned_TT/inputs_{innerstart}_to_{endindex}_with_default_{default}.npy', dataset)  
        np.save(f'/hpcwork/um106329/may_21/cleaned_TT/deepcsv_{innerstart}_to_{endindex}_with_default_{default}.npy', DeepCSV_dataset) 
    gc.collect()
    
```
Remember that we stored the start and end index at the beginning by making use of the user input with the argparser? Those two values will no be used to define the range of files to be used for the cleaning. The indices must be chosen such that they meet the available filepaths (for TT or QCD separately). More on this later. Assume you give the indices 0 and 99. Then, the loop will consist of two iterations, one between file 0 and 49, and the next between 50 and 99. This is to make sure one can combine several of the small files into one, but still maintaining reasonable "big-file" sizes that will later be used for the training etc.<br>
The code inside the loop first of all checks if the current innerstart index (the one that starts the chunk of 50 files) is *not the last innerstart index* (this is the check in line 272). That means that a full package of 50 files can be run over. Should this if-clause be False, one only has the remaining files left to be compared into one bigger file (this happens in the else case that starts at line 283). Other than that, the two cases are similar, in that they first ask if it's the first file in the chunk (i == 0) or not. The construction of the inner loop yields not only the indices of files n, but also the how manyth entry in the chunk this is (i). If it's the first in the chunk, a new combination of dataset and DeepCSV_dataset is created based on the cleandataset function given the current file path n from the inner loop. Should there already exist a preliminary dataset from the i==0 iteration, instead a new temporary dataset is created with the current i and n, and this is concatenated with the previous version of the big dataset. Once the inner loop is ready, our own dataset with inputs and targets will be stored, and the same is done with the DeepCSV outputs as well.<br>
The else-block from the outer loop is used wheneevr the specified range (the number of files for cleaning) between the start and end index is not divisible by 50. like if one would specifiy 0 to 80, first the if-part would run with files 0 to 49, and then the else-part with indices 50-(80-1), meaning including the files with indices 50,51,...,78,79 (not 80).<br>
As you can probably imagine, the file paths need to be modified to match your personal file system / the directory structure in your own hpcwork partition. hpcwork is used because it allows the storage of up to 1TB data, and has fast access. But note that hpcwork, unlike home, is not backed up.

## Applying the cleandataset function interactively <a name="interactive"></a>
If you want to compare the data before and after application of the cleandataset function on a small data set first, you can start by running the function interactively with one input file first, before moving to the processing of *every* file available in a following step.

### Manually check some jets before cleaning
Here is some skeleton to start investigating the input files. It's basically a modified version of the `clean.py` script. Make sure you have the grid-proxy activated, otherwise the files can't be read:
```
conda activate my-env

voms-proxy-init --voms cms --vomses ~/.grid-security/vomses
```
The following cells can be executed via `Shift+Enter`

In [ ]:
import uproot4 as uproot  # in case you installed uproot4 already under the name uproot (at some point, there was a renaming), just write import uproot
import awkward1 as ak
import numpy as np
import pandas                        # this is just for convenience when investigating the data, not necessary for clean.py
import matplotlib.pyplot as plt      # for plotting
import mplhep as hep                 # for plotting, but in a cool way ;-)
plt.style.use([hep.style.ROOT, hep.style.firamath])
import gc
import json

In [ ]:
list_paths = []
with open('/home/um106329/aisafety/new_march_21/qcd_file_paths.json') as json_file:
    json_paths = json.load(json_file)
    for entry in json_paths['TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8']:
        list_paths.append(entry)
        break  # one file is enough, so just stop the loop directly after the first iteration

In [ ]:
tree = uproot.open(list_paths[0], library='pd')['Events']  # the .root-file comes with the very interesting branch 'Events'
tree.keys()  # and this branch has again some tree structure, with many keys or features / variables...

In [ ]:
# you should recognize these feature names from the cleandataset function introduces above
feature_names = [k for k in tree.keys() if  (('Jet_eta' == k) or ('Jet_pt' == k) or ('Jet_DeepCSV' in k))]
# tagger output to compare with later and variables used to get the truth output
feature_names.extend(('Jet_btagDeepB_b','Jet_btagDeepB_bb', 'Jet_btagDeepC','Jet_btagDeepL'))
feature_names.extend(('Jet_nBHadrons', 'Jet_hadronFlavour'))

# grab the interesting arrays in a dataframe
df = tree.arrays(filter_name=feature_names, library="pd")

In [ ]:
df.head()  # first entry in the dataframe (= first event) comes with several subentries (= jets inside the first event)
# and for all of them, there are the different columns filled with values

In [ ]:
df.describe()  # get some basic statistics of each variable (more descriptive after the cleaning has been applied due to weird values or outliers at the beginning)

In [ ]:
# simple histogram of the jet pseudorapidity, as demonstrated here: https://gist.github.com/XavierAtCERN/4dd3d19482e271ff9a54125f0fa58b25
f, ax = plt.subplots()

ax.hist(df['Jet_eta'], bins=np.arange(-3., 3., .2), alpha = 0.5, label='Your first dataset')

plt.xlabel('Jet $\eta$')
plt.ylabel('Number of jets / 0.2 units')

plt.legend()
plt.show()

In [ ]:
# these are all features used for our studies (except the manually created targets, which come later)
feature_names

In [ ]:
# here you can experiment a bit with more histograms / plots of the other variables if you like
# use the feature_names to get the desired feature of the dataframe, specify the name directly or use indices,
# using indices could be convenient when plotting more than one variable, like so (even better would be defining a function for plotting):
for i in range(0,len(feature_names)):
    plt.hist(df[feature_names[i]], bins=100, log=True, alpha = 0.5, label='Your first dataset')
    plt.xlabel(feature_names[i])
    plt.ylabel("Jets")
    plt.legend()
    plt.show()
# if you find that the variable names look a bit annoying with the underscores, there is a file called "definitions.py" inside the june_21/evaluate directory
# which has names for displaying them, could be modified to your liking

Here are some tasks with which you can practice a bit:
- Are there some variables with 'weird' values? If so, what is the fraction of weird values? Do you see any pattern / are they correlated among different features?
- Try to compute the true flavour (b, bb, c, udsg) for all jets in the dataframe, add this as an additional column, plot a histogram of the targets (could be done purely in pandas)
    - hint: you can utilize the hadronFlavour and nBHadrons for this task similar to the definition inside the cleandataset function, just with another framework than how it was done there (in this case, here: pandas), or, if you want to do it differently, convert the dataframe into a format that's more convenient for you (e.g. numpy, where the syntax should be very similar to what you see inside cleandataset)
- Plot the inputs split by flavour (with the additional column, you just have to modify the plotting a bit) and explore the distributions.

In [ ]:
# your code here, if you want to practice pandas / numpy / matplotlib... and gain some insights into your data
# you can also skip this and move to the data cleaning directly

So you found out how the basic distributions of the input file look like, before any cleaning has been applied. You probably noticed some strange variables. The cleaning will hopefully end up with useful, physical distributions, although some defaults can not be excluded completely.

In [ ]:
# copy the cleandataset function here
def cleandataset(f):
    #print('Doing cleaning, isMC = ',isMC)
    feature_names = [k for k in f['Events'].keys() if  (('Jet_eta' == k) or ('Jet_pt' == k) or ('Jet_DeepCSV' in k))]
    # tagger output to compare with later and variables used to get the truth output
    feature_names.extend(('Jet_btagDeepB_b','Jet_btagDeepB_bb', 'Jet_btagDeepC','Jet_btagDeepL'))
    feature_names.extend(('Jet_nBHadrons', 'Jet_hadronFlavour'))
    #print(feature_names)
    #print(len(feature_names))
    
    # go through a specified number of events, and get the information (awkward-arrays) for the keys specified above
    for data in f['Events'].iterate(feature_names, step_size=f['Events'].num_entries, library='ak'):
        break
        
    
    # creating an array to store all the columns with their entries per jet, flatten per-event -> per-jet
    datacolumns = np.zeros((len(feature_names)+1, len(ak.flatten(data['Jet_pt'], axis=1))))
    #print(len(datacolumns))

    for featureindex in range(len(feature_names)):
        a = ak.flatten(data[feature_names[featureindex]], axis=1) # flatten along first inside to get jets
        
        datacolumns[featureindex] = ak.to_numpy(a)

    nbhad = ak.to_numpy(ak.flatten(data['Jet_nBHadrons'], axis=1))
    hadflav = ak.to_numpy(ak.flatten(data['Jet_hadronFlavour'], axis=1))

    target_class = np.full_like(hadflav, 3)                                                      # udsg
    target_class = np.where(hadflav == 4, 2, target_class)                                       # c
    target_class = np.where(np.bitwise_and(hadflav == 5, nbhad > 1), 1, target_class)            # bb
    target_class = np.where(np.bitwise_and(hadflav == 5, nbhad <= 1), 0, target_class)           # b, lepb

    #print(np.unique(target_class))

    #datacolumns[len(feature_names)] = ak.to_numpy(target_class)
    datacolumns[len(feature_names)] = target_class
    #print(np.unique(datacolumns[len(feature_names)]))
        
    datavectors = datacolumns.transpose()
    #print(np.unique(datavectors[:,len(feature_names)]))
    
    #print(i)
    for j in range(67):
        datavectors[:, j][datavectors[:, j] == np.nan]  = defaults_per_variable[j]
        datavectors[:, j][datavectors[:, j] <= -np.inf] = defaults_per_variable[j]
        datavectors[:, j][datavectors[:, j] >= np.inf]  = defaults_per_variable[j]
        datavectors[:, j][datavectors[:, j] == -999]  = defaults_per_variable[j]  # this one line is new and the reason for that is that there can be "original" -999 defaults in the inputs that should now also move into the new
                                                               # default bin, it was not necessary in my old clean_1_2.py code, because I could just leave them where they are, here they need to to be modified
        #print(np.unique(datavectors[:,-1]))
    #print(np.unique(datavectors[:,-1]))
    datavecak = ak.from_numpy(datavectors)
    #print(ak.unique(datavecak[:,-1]))
    #print(len(datavecak),"entries before cleaning step 1")
    
    # cutting stuff away only needs to be done for train, val, test without ScaleFactor framework, there these can just be commented out
    # tracker acceptance and pt range
    datavecak = datavecak[datavecak[:, 0] >= -2.5]
    datavecak = datavecak[datavecak[:, 0] <= 2.5]
    datavecak = datavecak[datavecak[:, 1] >= 20.]
    datavecak = datavecak[datavecak[:, 1] <= 1000.]
    datavecak = datavecak[datavecak[:, 57] <= 0.3]
    #datavecak = datavecak[datavecak[:, 67] >= 0.]
    #datavecak = datavecak[datavecak[:, 67] <= 1.]
    #datavecak = datavecak[datavecak[:, 68] >= 0.]
    #datavecak = datavecak[datavecak[:, 68] <= 1.]
    #datavecak = datavecak[datavecak[:, 69] >= 0.]
    #datavecak = datavecak[datavecak[:, 69] <= 1.]
    #datavecak = datavecak[datavecak[:, 70] >= 0.]
    #datavecak = datavecak[datavecak[:, 70] <= 1.]

    

    # check jetNSelectedTracks, jetNSecondaryVertices > 0
    #datavecak = datavecak[(datavecak[:, 63] > 0) | (datavecak[:, 64] > 0)]  # keep those where at least any of the two variables is > 0, they don't need to be > 0 simultaneously
    #print(len(datavecak),"entries after cleaning step 1")

    alldata = ak.to_numpy(datavecak)
    #print(np.unique(alldata[:,-1]))
        
    
    for track0_vars in [6,12,22,29,35,42,50]:
        alldata[:,track0_vars][alldata[:,64] <= 0] = defaults_per_variable[track0_vars]
    for track0_1_vars in [7,13,23,30,36,43,51]:
        alldata[:,track0_1_vars][alldata[:,64] <= 1] = defaults_per_variable[track0_1_vars]
    for track01_2_vars in [8,14,24,31,37,44,52]:
        alldata[:,track01_2_vars][alldata[:,64] <= 2] = defaults_per_variable[track01_2_vars]
    for track012_3_vars in [9,15,25,32,38,45,53]:
        alldata[:,track012_3_vars][alldata[:,64] <= 3] = defaults_per_variable[track012_3_vars]
    for track0123_4_vars in [10,16,26,33,39,46,54]:
        alldata[:,track0123_4_vars][alldata[:,64] <= 4] = defaults_per_variable[track0123_4_vars]
    for track01234_5_vars in [11,17,27,34,40,47,55]:
        alldata[:,track01234_5_vars][alldata[:,64] <= 5] = defaults_per_variable[track01234_5_vars]
    alldata[:,18][alldata[:,65] <= 0] = defaults_per_variable[18]
    alldata[:,19][alldata[:,65] <= 1] = defaults_per_variable[19]
    alldata[:,20][alldata[:,65] <= 2] = defaults_per_variable[20]
    alldata[:,21][alldata[:,65] <= 3] = defaults_per_variable[21]

    for AboveCharm_vars in [41,48,49,56]:
        alldata[:,AboveCharm_vars][alldata[:,AboveCharm_vars]==-1] = defaults_per_variable[AboveCharm_vars] 
    
    
    datacls = [i for i in range(0,67)]
    datacls.append(73)
    dataset = alldata[:, datacls]
     
    DeepCSV_dataset = alldata[:, 67:71]
    
    return dataset, DeepCSV_dataset

In [ ]:
# define a global default that can be subtracted from the minima:
default = 0.001
# load minima, but make sure to use your own paths
minima = np.load('/home/um106329/aisafety/april_21/from_Nik/default_value_studies_minima.npy')
defaults_per_variable = minima - default

In [ ]:
# run the cleaning and store the results
dataset, DeepCSV_dataset = cleandataset(uproot.open(list_paths[0]))

In [ ]:
# make basic histograms with cleaned data
# inputs
for i in range(67):
    plt.hist(dataset[:,i], bins=100, log=True)
    plt.xlabel(feature_names[i])
    plt.ylabel('Jets')
    plt.show()

In [ ]:
# targets
plt.hist(dataset[:,-1], bins=100, log=True)
plt.xlabel('True flavour category')
plt.ylabel('Jets')

In [ ]:
# DeepCSV
categories = ['b', 'bb', 'c', 'udsg']
for i in range(4):
    plt.hist(DeepCSV_dataset[:,i], bins=100, log=True)
    plt.xlabel('P('+categories[i]+')')
    plt.ylabel('Jets')
    plt.show()

Tasks if you want to explore the data a bit further:
- The histograms could be split by flavour, here it's much easier because we already have that column as a result from the cleaning.
- Do you notice some differences between the cleaned and original distributions? I'm thinking about checking the correlations or doing some scatter plots to investigate the effect of the cleaning.

In [ ]:
# room for your code

Another task (not really necessary when starting): if you want to compute the discriminators, use the formulae
$$
BvsL = \frac{P(b)+P(bb)}{P(b)+P(bb)+P(udsg)}
$$

$$
BvsC = \frac{P(b)+P(bb)}{P(b)+P(bb)+P(c)}
$$

$$
CvsB = \frac{P(c)}{P(b)+P(bb)+P(c)}
$$

$$
CvsL = \frac{P(c)}{P(c)+P(udsg)}
$$

for some additional arrays and then plot also the resulting discriminator shapes. Exclude the defaults, or you will end up with some strange spikes. Also note that in some cases, if some probabilities are exactly 1, the calculation could be erronous --> make sure that dividing by zero is handled correctly, should there be any error (warnings can be ignored). Theoretically, the discriminators are also part of the input files, but we do not store them explicitly here, as they can be reconstructed from what we have saved. Again it could be interesting to split this by flavour.

In [ ]:
# probably some code to check the discriminators

## Practical data cleaning @HPC <a name="atHPC"></a>
Now it's time to actually do the data cleaning systematically, after all that text and finally also with all files. :-)

### Create directories
In order to store the cleaned data "permanently", i.e. without having to run the cleaning step every time, you need some directories where to put the cleaned data. Take for example something like
```
mkdir /hpcwork/yourtimID/october_21/cleaned_TT
mkdir /hpcwork/yourtimID/october_21/cleaned_QCD
```
to store the arrays that result from the cleaning step, so the arrays for our own tagger and the arrays that contain DeepCSV outputs.

### Login to (potentially many) interactive nodes and activate the grid-proxy
The script inside `clean.py` currently only runs interactively, simply because so far, I did not find out how to use the grid-proxy inside SLURM batch jobs, unlike HTCondor, where one can ship the proxy with the job. It's perfectly possible that I just didn't find the right instructions for SLURM and that it works there as well. Anyway, with the current setup, we need to call the cleaning script from the dialog systems. And they come with a caveat: "Processes that have consumed more than 20 minutes CPU time may get killed without warning." [see documentation](https://help.itc.rwth-aachen.de/service/rhr4fjjutttf/article/0a23d513f31b4cf1849986aaed475789/).

So to "cheat" a little without having to wait *forever*, one can just login to many interactive nodes at a time and split the cleaning into several small units that themselves stay within the interactive time limit, and distribute the work over more than one node. The "good" I/O-nodes are the [copy...-nodes, which are optimized for data transfer](https://help.itc.rwth-aachen.de/service/rhr4fjjutttf/article/dbd3240c521d4595bec6b08667c4d42c/), but also the other standard nodes or even the -g-nodes can be used.

Activate the grid-proxy on each interactive node from within the right conda environment
```
conda activate my-env

voms-proxy-init --voms cms --vomses ~/.grid-security/vomses
```
to perform the cleaning as explained below.

### Execute the data cleaning for both processes (TT and QCD)
Start for example by cleaning the semileptonic tt̅ events. Make sure all paths in the cleaning script are correct and that they point to the TT-directories of your personal hpcwork file system. The ranges used for the file paths are
```
0-499,500-999,1000-1499,1500-1999,2000-2446
```
meaning packages of 500 files, except for the last one, which has fewer files.<br>
Note that for now, the tested default value is 0.001, that means on the first node you would start the cleaning with a command like this one:
```
python clean.py 0 499 0.001
```
and on another node, you'd do something similar, but with the next chunk of files (500 to 999), and so on until you have cleaned all semileptonic tt̅ events. This can already take some time, even if distributed among the available nodes.

When ready with the semileptonic tt̅ events, the steps should be repeated with the QCD multijet samples as well. But before this can be done, the code needs to be changed for this new purpose: lines 32 to 46 should not be commented anymore, but the semileptonic tt̅ part has to be commented out (that means you basically switch the comment block with the three ''' so that it's exactly opposite now).<br>
For lines 281 and 282, as well as lines 292 and 293, replace the `_TT` with `_QCD` to match the directory structure you created above.<br>
For QCD files, the ranges are
```
0-499,500-999,and so on,11000-11407
```
which means that for QCD, you will need even more nodes (or wait longer) and have to perform a couple of rounds until all files are finally cleaned. I know it's not very convenient, but if it takes long, you can be quite sure you are doing the right thing. ;-)

So congrats, the part which is the least convenient is done, and you will not need the grid-proxy anymore, unless to want to modify the cleaning later on with new ideas.